<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-sample-dates-and-population" data-toc-modified-id="Load-sample-dates-and-population-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load sample dates and population</a></span></li><li><span><a href="#Load-Fact-Data" data-toc-modified-id="Load-Fact-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Fact Data</a></span></li><li><span><a href="#Create-Feature-Sets" data-toc-modified-id="Create-Feature-Sets-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create Feature Sets</a></span></li><li><span><a href="#Test-FeatureTool-Related-Functions" data-toc-modified-id="Test-FeatureTool-Related-Functions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test FeatureTool Related Functions</a></span><ul class="toc-item"><li><span><a href="#Create-Entity-and-Feature-Set" data-toc-modified-id="Create-Entity-and-Feature-Set-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Create Entity and Feature Set</a></span></li></ul></li><li><span><a href="#Put-everything-together:-main()" data-toc-modified-id="Put-everything-together:-main()-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Put everything together: main()</a></span></li><li><span><a href="#Appendix" data-toc-modified-id="Appendix-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Appendix</a></span><ul class="toc-item"><li><span><a href="#Documentation-Reduced-Feature-Set" data-toc-modified-id="Documentation-Reduced-Feature-Set-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Documentation Reduced Feature Set</a></span></li><li><span><a href="#Documentation-NaN-Handling" data-toc-modified-id="Documentation-NaN-Handling-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Documentation NaN Handling</a></span></li></ul></li></ul></div>

<div class='alert alert-block alert-info'>
<b>Note:</b> This nb is for testing of the final pipeline for generating a full "fact feature" set using feature tools (based on insights from nb 1)
</div>


In [1]:
import sys
from datetime import date, datetime, timedelta
from pathlib import Path

import bcag
import featuretools as ft
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bcag.sql_utils import execute_sql_query
from sqlalchemy.engine.base import Engine

In [3]:
sys.path.append(str(Path.cwd().parent))
from churn21.data import dates as utils_dt, load as utils_ld, fact_features as utils_ff

In [4]:
%load_ext autoreload
%autoreload 1

%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
print(sys.executable)
print(sys.version)
print(f'Pandas {pd.__version__}')

W:\conda-environments\churn21\python.exe
3.9.4 (default, Apr  9 2021, 11:43:21) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.4


In [6]:
engine = bcag.connect("jemas", "prod", "jemas_temp")

## Load sample dates and population

In [7]:
dt_params = dict({
    "last_cut_off_date_train": date(2020, 10, 31),
    "first_cut_off_date_train": date(2020, 7, 31),
    "lookback_period_months": 13,
    "label_period_months": 3,
    "n_months_considered_training": 6
})

l_dates_train, l_dates_test = utils_dt.create_dateinfo(dt_params)

# Check dates for first training cohort
l_dates_train

[{'dt_cut_off': datetime.date(2020, 7, 31),
  'dt_obs_first_considered': datetime.date(2019, 7, 1),
  'dt_label_last_considered': datetime.date(2020, 10, 31)},
 {'dt_cut_off': datetime.date(2020, 8, 31),
  'dt_obs_first_considered': datetime.date(2019, 8, 1),
  'dt_label_last_considered': datetime.date(2020, 11, 30)},
 {'dt_cut_off': datetime.date(2020, 9, 30),
  'dt_obs_first_considered': datetime.date(2019, 9, 1),
  'dt_label_last_considered': datetime.date(2020, 12, 31)},
 {'dt_cut_off': datetime.date(2020, 10, 31),
  'dt_obs_first_considered': datetime.date(2019, 10, 1),
  'dt_label_last_considered': datetime.date(2021, 1, 31)}]

In [8]:
first_date = l_dates_train[0]["dt_obs_first_considered"]
cut_off_date = l_dates_train[0]["dt_cut_off"]

In [9]:
df_population = utils_ld.load_population(engine)
df_population = utils_ld.filter_population(df_population, cut_off_date)

assert df_population.duplicated().sum() == 0

df_population.shape
df_population.head()

(134017, 1)

,konto_lauf_id
0,51
1,129
2,157
3,235
4,503


## Load Fact Data

In [14]:
df_sales_fact = utils_ff.load_sales_fact(cut_off_date, first_date, engine)

In [15]:
df_fees_fact = utils_ff.load_fees_fact(cut_off_date, first_date, engine)

In [17]:
# Safety check - should be roughly one week later than cut_off_date

assert df_sales_fact["kauf_datum"].max() > cut_off_date
assert df_fees_fact["kauf_datum"].max() > cut_off_date

In [29]:
# # Save to parquet file for faster testing

# rel_path = 'test_data_raph'
# if not Path(rel_path).exists():
#     Path(rel_path).mkdir()
    
# df_sales_fact.to_parquet(Path(rel_path) / "df_sales_fact", index=False)
# df_fees_fact.to_parquet(Path(rel_path) / "df_fees_fact", index=False)

## Create Feature Sets

In [21]:
df_sales_fact_red = utils_ff.fit_fact_df_to_population(df_sales_fact, df_population)
sales_first, sales_12m, sales_last = utils_ff.split_fact_df_into_3_periods(df_sales_fact_red, cut_off_date, first_date)

In [24]:
# Check results 
for df in [sales_first, sales_12m, sales_last]:
    print(df.shape, df["kauf_datum"].min().date(), df["kauf_datum"].max().date())

(965194, 7) 2019-07-01 2019-07-31
(10372554, 7) 2019-08-01 2020-07-31
(1014821, 7) 2020-07-01 2020-07-31


In [26]:
df_fees_fact_red = utils_ff.fit_fact_df_to_population(df_fees_fact, df_population)
fees_first, fees_12m, fees_last = utils_ff.split_fact_df_into_3_periods(df_fees_fact_red, cut_off_date, first_date)

In [30]:
# Check results 
for df in [fees_first, fees_12m, fees_last]:
    print(df.shape, df["kauf_datum"].min().date(), df["kauf_datum"].max().date())

(1018401, 5) 2019-07-01 2019-07-31
(9281148, 5) 2019-08-01 2020-07-31
(831163, 5) 2020-07-01 2020-07-31


## Test FeatureTool Related Functions

### Create Entity and Feature Set

In [77]:
%aimport churn21.data.fact_features

In [40]:
es_first = utils_ff.create_entity_set_ft(df_population, sales_first, fees_first)
es_12m = utils_ff.create_entity_set_ft(df_population, sales_12m, fees_12m )
es_last = utils_ff.create_entity_set_ft(df_population, sales_last, fees_last)

In [41]:
feature_matrix_first, _ = utils_ff.create_ft_matrix_and_defs_reduced(es_first, cut_off_date, n_jobs=-1)

Built 12 features
EntitySet scattered to 8 workers in 33 seconds
Elapsed: 00:29 | Progress: 100%|██████████


In [43]:
feature_matrix_last, _ = utils_ff.create_ft_matrix_and_defs_reduced(es_last, cut_off_date, n_jobs=-1)

Built 12 features
EntitySet scattered to 8 workers in 57 seconds
Elapsed: 00:28 | Progress: 100%|██████████


In [44]:
# Safety check
assert feature_matrix_first.shape == feature_matrix_last.shape

In [98]:
feature_matrix_12m, _ = utils_ff.create_ft_matrix_and_defs_full(es_12m, cut_off_date, n_jobs=1)

Built 46 features
Elapsed: 40:33 | Progress: 100%|██████████


In [99]:
# Safety Copy
feature_matrix_12m.to_csv("fm_12m.csv", index=True)

# Check result
feature_matrix_12m.shape

(134017, 46)

In [109]:
# Handle Missing Values

fm_12m = utils_ff.impute_missing_values_full(feature_matrix_12m)
fm_first = utils_ff.impute_missing_values_reduced(feature_matrix_first)
fm_last = utils_ff.impute_missing_values_reduced(feature_matrix_last)

In [111]:
# Check all cols with NaN are either MODE, SKEW or TREND
[col for col in fm_12m.columns if fm_12m[col].isna().sum() > 0]

['MODE(sales_fact.mcg_id)',
 'MODE(sales_fact.transaction_type_id)',
 'MODE(sales_fact.transaktionsart_id_korr)',
 'SKEW(sales_fact.betrag)',
 'TREND(sales_fact.betrag, kauf_datum)',
 'MODE(fees_fact.bewegungstyp)',
 'SKEW(fees_fact.betrag)',
 'TREND(fees_fact.betrag, kauf_datum)',
 'TREND(sales_fact.betrag, kauf_datum WHERE transaktionsart_id_korr = 0)',
 'TREND(fees_fact.betrag, kauf_datum WHERE bewegungstyp = fremdw)',
 'TREND(fees_fact.betrag, kauf_datum WHERE bewegungstyp = mahnung)',
 'TREND(fees_fact.betrag, kauf_datum WHERE bewegungstyp = zins)']

In [113]:
trend_features = utils_ff.calculate_trend_last_minus_first(fm_first, fm_last)

# Safety Checks
assert fm_first.shape == fm_last.shape == trend_features.shape
assert fm_first.index.all() == fm_last.index.all() == trend_features.index.all()
assert pd.concat([fm_first, fm_last, trend_features], axis=1).isna().sum().sum() == 0

# Check results
trend_features.head()

,trend_AVG_TIME_BETWEEN(sales_fact.kauf_datum),trend_COUNT(sales_fact),trend_NUM_UNIQUE(sales_fact.mcg_id),trend_NUM_UNIQUE(sales_fact.transaction_type_id),trend_SUM(sales_fact.betrag),trend_COUNT(fees_fact),trend_NUM_UNIQUE(fees_fact.bewegungstyp),trend_SUM(fees_fact.betrag),trend_COUNT(sales_fact WHERE transaktionsart_id_korr = 0),trend_SUM(fees_fact.betrag WHERE bewegungstyp = fremdw),trend_SUM(fees_fact.betrag WHERE bewegungstyp = mahnung),trend_SUM(fees_fact.betrag WHERE bewegungstyp = zins)
konto_lauf_id,,,,,,,,,,,,
51,0.00,1.00,1.00,1.00,189.60,0.00,0.00,0.00,1.00,0.00,0.00,0.00
129,0.00,1.00,-1.00,0.00,-182.10,-18.00,-2.00,-62.44,0.00,-60.43,0.00,0.00
157,"41,792.48",-2.00,0.00,0.00,760.02,-10.00,-1.00,-26.80,0.00,-24.79,0.00,0.00
235,"2,678,040.00",-2.00,-1.00,-1.00,-134.90,0.00,0.00,-3.90,-2.00,0.00,0.00,-3.91
503,"-282,240.00",9.00,1.00,1.00,"2,408.40",15.00,1.00,59.49,0.00,59.47,0.00,0.00


In [117]:
# Concat to final fact feature set
full_fact_feature_set_first = utils_ff.concat_fm_12m_and_trend_features(fm_12m, trend_features)

# Pass tests
assert full_fact_feature_set_first.shape[1] == (
    feature_matrix_12m.shape[1] 
    + trend_features.shape[1]
) 
assert (
    full_fact_feature_set_first.shape[0] 
    == feature_matrix_first.shape[0] 
    == trend_features.shape[0]
) 
assert (full_fact_feature_set_first.index == feature_matrix_first.index).all()

## Put everything together: main()

**Args for main function:** pop, cut_off_date, first_date, engine, n_jobs

In [82]:
def main(pop, cut_off_date, first_date, engine, n_jobs):
    # Load, fit and split fact data
    sales = utils_ff.load_sales_fact(cut_off_date, first_date, engine)
    sales_red = utils_ff.fit_fact_df_to_population(sales, df_population)
    sales_first, sales_12m, sales_last = utils_ff.split_fact_df_into_3_periods(sales_red, cut_off_date, first_date)
    
    fees = utils_ff.load_fees_fact(cut_off_date, first_date, engine)
    fees_red = utils_ff.fit_fact_df_to_population(fees, df_population)
    fees_first, fees_12m, fees_last = utils_ff.split_fact_df_into_3_periods(fees_red, cut_off_date, first_date)
    
    # Create entity sets and featuretools matrices
    es_first = utils_ff.create_entity_set_ft(df_population, sales_first, fees_first)
    es_last = utils_ff.create_entity_set_ft(df_population, sales_last, fees_last)
    es_12m = utils_ff.create_entity_set_ft(df_population, sales_12m, fees_12m )
    
    fm_first, _ = utils_ff.create_ft_matrix_and_defs_reduced(es_first, cut_off_date, n_jobs)
    fm_last, _ = utils_ff.create_ft_matrix_and_defs_reduced(es_last, cut_off_date, n_jobs)
#     fm_12m, _ = utils_ff.create_ft_matrix_and_defs_full(es_12m, cut_off_date, n_jobs)
    
    fm_first = utils_ff.impute_missing_values_reduced(fm_first)
    fm_last = utils_ff.impute_missing_values_reduced(fm_last)
    
    # Calculate trend_features and append to full set
    trend_features = utils_ff.calculate_trend_last_minus_first(fm_first, fm_last)
    df_ft = utils_ff.concat_fm_12m_and_trend_features(fm_12m, trend_features)
    
#     return df_ft

    return fm_first, fm_last, trend_features

In [83]:
fm_first, fm_last, trend_features = main(df_population, cut_off_date, first_date, engine, n_jobs=-1)

Built 12 features
EntitySet scattered to 8 workers in 33 seconds
Elapsed: 00:33 | Progress: 100%|██████████
Built 12 features
EntitySet scattered to 8 workers in 30 seconds
Elapsed: 00:28 | Progress: 100%|██████████


In [90]:
assert fm_first.shape == fm_last.shape == trend_features.shape
assert fm_first.index.all() == fm_last.index.all() == trend_features.index.all()
assert pd.concat([fm_first, fm_last, trend_features], axis=1).isna().sum().sum() == 0

In [94]:
fm_first.describe()
fm_last.describe()
trend_features.describe()

,AVG_TIME_BETWEEN(sales_fact.kauf_datum),COUNT(sales_fact),NUM_UNIQUE(sales_fact.mcg_id),NUM_UNIQUE(sales_fact.transaction_type_id),SUM(sales_fact.betrag),COUNT(fees_fact),NUM_UNIQUE(fees_fact.bewegungstyp),SUM(fees_fact.betrag),COUNT(sales_fact WHERE transaktionsart_id_korr = 0),SUM(fees_fact.betrag WHERE bewegungstyp = fremdw),SUM(fees_fact.betrag WHERE bewegungstyp = mahnung),SUM(fees_fact.betrag WHERE bewegungstyp = zins)
count,"134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00"
mean,"1,287,151.08",7.20,1.60,1.26,565.38,7.60,1.33,20.25,0.12,8.39,0.64,8.29
std,"1,180,882.83",12.36,1.55,1.12,"1,073.28",14.17,1.06,67.78,0.64,23.80,3.52,17.92
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,"186,092.31",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,"561,600.00",3.00,1.00,1.00,161.94,3.00,1.00,4.83,0.00,0.00,0.00,0.00
75%,"2,678,400.00",9.00,2.00,2.00,668.85,9.00,2.00,27.66,0.00,5.59,0.00,7.46
max,"2,678,400.00",233.00,12.00,6.00,"61,003.63",384.00,4.00,"11,034.50",33.00,"1,338.11",20.00,"1,630.45"


,AVG_TIME_BETWEEN(sales_fact.kauf_datum),COUNT(sales_fact),NUM_UNIQUE(sales_fact.mcg_id),NUM_UNIQUE(sales_fact.transaction_type_id),SUM(sales_fact.betrag),COUNT(fees_fact),NUM_UNIQUE(fees_fact.bewegungstyp),SUM(fees_fact.betrag),COUNT(sales_fact WHERE transaktionsart_id_korr = 0),SUM(fees_fact.betrag WHERE bewegungstyp = fremdw),SUM(fees_fact.betrag WHERE bewegungstyp = mahnung),SUM(fees_fact.betrag WHERE bewegungstyp = zins)
count,"134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00"
mean,"1,284,517.59",7.57,1.54,1.27,548.46,6.20,1.26,17.13,0.09,5.55,0.77,7.25
std,"1,177,657.24",13.06,1.47,1.12,"1,065.29",11.78,1.05,102.75,0.59,19.04,3.85,15.87
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,"190,080.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,"576,000.00",3.00,1.00,1.00,149.90,2.00,1.00,3.00,0.00,0.00,0.00,0.00
75%,"2,678,400.00",9.00,2.00,2.00,635.54,7.00,2.00,21.81,0.00,2.90,0.00,5.12
max,"2,678,400.00",310.00,12.00,6.00,"56,631.72",616.00,4.00,"17,500.00",35.00,"1,422.97",20.00,457.68


,trend_AVG_TIME_BETWEEN(sales_fact.kauf_datum),trend_COUNT(sales_fact),trend_NUM_UNIQUE(sales_fact.mcg_id),trend_NUM_UNIQUE(sales_fact.transaction_type_id),trend_SUM(sales_fact.betrag),trend_COUNT(fees_fact),trend_NUM_UNIQUE(fees_fact.bewegungstyp),trend_SUM(fees_fact.betrag),trend_COUNT(sales_fact WHERE transaktionsart_id_korr = 0),trend_SUM(fees_fact.betrag WHERE bewegungstyp = fremdw),trend_SUM(fees_fact.betrag WHERE bewegungstyp = mahnung),trend_SUM(fees_fact.betrag WHERE bewegungstyp = zins)
count,"134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00","134,017.00"
mean,"-2,633.49",0.37,-0.06,0.01,-16.92,-1.40,-0.07,-3.12,-0.02,-2.84,0.13,-1.03
std,"1,119,961.14",10.53,1.37,1.02,"1,029.29",14.19,1.01,118.91,0.62,25.54,4.87,11.57
min,"-2,678,400.00",-176.00,-8.00,-6.00,"-48,081.44",-349.00,-4.00,"-10,933.63",-31.00,-899.37,-20.00,"-1,630.45"
25%,"-93,960.00",-2.00,-1.00,0.00,-185.88,-3.00,-1.00,-7.71,0.00,-1.96,0.00,0.00
50%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,"95,040.00",2.00,1.00,0.00,166.20,1.00,0.00,2.00,0.00,0.00,0.00,0.00
max,"2,678,400.00",310.00,8.00,5.00,"28,554.00",616.00,4.00,"17,499.17",21.00,"1,157.66",20.00,309.52


---

## Appendix 

### Documentation Reduced Feature Set

12 features (out of 46 in total) used for trend calculation month_last - month_first

In [ ]:
# Dokumentation of selection of the trend features

AVG_TIME_BETWEEN(sales_fact.kauf_datum),
COUNT(sales_fact),
# MAX(sales_fact.betrag),
# MEAN(sales_fact.betrag),
# MIN(sales_fact.betrag),
# MODE(sales_fact.mcg_id),
# MODE(sales_fact.transaction_type_id),
# MODE(sales_fact.transaktionsart_id_korr),
NUM_UNIQUE(sales_fact.mcg_id),
NUM_UNIQUE(sales_fact.transaction_type_id),
# NUM_UNIQUE(sales_fact.transaktionsart_id_korr),
# SKEW(sales_fact.betrag),
# STD(sales_fact.betrag),
SUM(sales_fact.betrag),
# TIME_SINCE_LAST(sales_fact.kauf_datum),
# TREND(sales_fact.betrag, kauf_datum),
# AVG_TIME_BETWEEN(fees_fact.kauf_datum),
COUNT(fees_fact),
# MAX(fees_fact.betrag),
# MEAN(fees_fact.betrag),
# MIN(fees_fact.betrag),
# MODE(fees_fact.bewegungstyp),
NUM_UNIQUE(fees_fact.bewegungstyp),
# SKEW(fees_fact.betrag),
# STD(fees_fact.betrag),
SUM(fees_fact.betrag),
# TIME_SINCE_LAST(fees_fact.kauf_datum),
# TREND(fees_fact.betrag, kauf_datum),
COUNT(sales_fact WHERE transaktionsart_id_korr = 0),
# MEAN(sales_fact.betrag WHERE transaktionsart_id_korr = 0),
# NUM_UNIQUE(sales_fact.MONTH(kauf_datum)),
# SUM(sales_fact.betrag WHERE transaktionsart_id_korr = 0),
# TREND(sales_fact.betrag, kauf_datum WHERE transaktionsart_id_korr = 0),
# COUNT(fees_fact WHERE bewegungstyp = fremdw),
# COUNT(fees_fact WHERE bewegungstyp = mahnung),
# COUNT(fees_fact WHERE bewegungstyp = zins),
# MEAN(fees_fact.betrag WHERE bewegungstyp = fremdw),
# MEAN(fees_fact.betrag WHERE bewegungstyp = mahnung),
# MEAN(fees_fact.betrag WHERE bewegungstyp = zins),
# NUM_UNIQUE(fees_fact.MONTH(kauf_datum)),
SUM(fees_fact.betrag WHERE bewegungstyp = fremdw),
SUM(fees_fact.betrag WHERE bewegungstyp = mahnung),
SUM(fees_fact.betrag WHERE bewegungstyp = zins),
# TREND(fees_fact.betrag, kauf_datum WHERE bewegungstyp = fremdw),
# TREND(fees_fact.betrag, kauf_datum WHERE bewegungstyp = mahnung),
# TREND(fees_fact.betrag, kauf_datum WHERE bewegungstyp = zins)

### Documentation NaN Handling

**Strategy:**

- "sum" -> [no NaN] 
- "count" -> [no NaN]


- "max" -> 0
- "min" -> 0
- "mean" -> 0
- "num_unique" -> 0


- "avg_time_between", -> max possible value in secs
- "time_since_last", -> max possible value in secs


- "std" ---- sklearn pipeline
- "mode" ---- sklearn pipeline
- "skew" ---- ? sklearn pipeline
- "trend" ---- ? sklearn pipeline

"avg_time_between" and "time_since_last" are returned in seconds by default


In [124]:
# Check which primitives do not generate NaN
for index, col in feature_matrix_12m.isna().sum().iteritems():
    if col == 0:
        print(index)

COUNT(sales_fact)
SUM(sales_fact.betrag)
COUNT(fees_fact)
SUM(fees_fact.betrag)
COUNT(sales_fact WHERE transaktionsart_id_korr = 0)
SUM(sales_fact.betrag WHERE transaktionsart_id_korr = 0)
COUNT(fees_fact WHERE bewegungstyp = fremdw)
COUNT(fees_fact WHERE bewegungstyp = mahnung)
COUNT(fees_fact WHERE bewegungstyp = zins)
SUM(fees_fact.betrag WHERE bewegungstyp = fremdw)
SUM(fees_fact.betrag WHERE bewegungstyp = mahnung)
SUM(fees_fact.betrag WHERE bewegungstyp = zins)
